# Исследование надежности заемщиков


## Откроем таблицу и изучим общую информацию о данных

**1. Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [1]:
import pandas as pd
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')


**2. Выведите первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


** 3. Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**4. Выведем количество пропущенных значений для каждого столбца.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**7. Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

**8. Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**9. В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**10. Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

**11. Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**З12. Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**13. Заменем вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**14. Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверем остальные столбцы.**

In [14]:
data['education'] = data['education'].str.lower()

**15. Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**16. На основании диапазонов, указанных ниже, создим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используя функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**17. Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**18. Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут нам правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуем данные и ответим на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Сгруппируем данные по количеству детей, посчитаем общее число заемщиков по каждой группе и количество заемщиков, кто не вернул в срок кредит.

In [22]:
data_cut=data.groupby('children')['debt'].agg(['count', 'sum']).rename(columns= {'count':'count_of_borrowers','sum':'not_returned'})

Добавим в сводную таблицу столбик где буден виден процен таких заемщиков от общего числа

In [23]:
data_cut['percent'] = round((data_cut['not_returned']/data_cut['count_of_borrowers']*100).astype('float'),2)
data_cut.sort_values(by='percent')

,count_of_borrowers,not_returned,percent
children,,,
5,9,0,0.00
0,14091,1063,7.54
3,330,27,8.18
1,4808,444,9.23
2,2052,194,9.45
4,41,4,9.76


**Вывод:** 

- Бездетные клиенты отдают кредиты на примерно 2 % лучше по сравнению с клиентами с одним или двумя детьми.

- Клиенты с одним или двумя детьми находятся на одном уровне по количеству задолженностей.

- Выборка по клиентам с 5 детьми слишком мала для оценки.

- Самый высокий процент задолженностей имеют клиенты с четырьмя детьми.


#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сгруппируем данные по статусу, посчитаем общее число заемщиков по статусу семейного положения клиента и количество заемщиков, кто не вернул в срок кредит.

In [24]:
data_family=data.groupby('family_status')['debt'].agg(['count', 'sum']).rename(columns= {'count':'count_status','sum':'not_returned'})

Добавим в сводную таблицу столбик где буден виден процен таких заемщиков от общего числа

In [25]:
data_family['percent'] = round((data_family['not_returned']/data_family['count_status']*100).astype('float'),2)
data_family.sort_values(by='percent')

,count_status,not_returned,percent
family_status,,,
вдовец / вдова,951,63,6.62
в разводе,1189,84,7.06
женат / замужем,12261,927,7.56
гражданский брак,4134,385,9.31
Не женат / не замужем,2796,273,9.76


**Вывод:** 

 - Клиенты в категории вдовец / вдова имеют самый низкий процент просрочек. Далее на пол процента больше просрочек у клиентов в категории в разводе.
 

 - У клиентов состоящих в официальном браке примерно такой же процент. 
 

 - Клиенты не состоящие в официальном браке (гражданский брак или не женат/не замужем) имеют самый высокий процент по не возврату кредита.
 
Данные показывают не большую, но все же разницу в возврате кредита в срок между клиентами:

Клиенты состоящие в официальном браке в настоящий момент или в прошлом  примерно на 2-3%  меньше задолженностей, чем клиенты в гражданском браке или не в браке.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создадим новую таблицу где будут копии колонок с данными о задолжности и доходом клиентов.

In [26]:
total_income=data.loc[:, ['total_income','debt']]
total_income.head()

,total_income,debt
0,253875,0
1,112080,0
2,145885,0
3,267628,0
4,158616,0


Разделим доход клиентов на 4 категории по значениям процентилей:

- низкий - доход не превышающий 25% 

- средний - в диапазоне от 25% до 50%

- высокий - в диапазоне от 50% до 75%

- очень высокий - превышающий 75% 

посмотрим размер процентилей, выведем на экран:

In [27]:
total_income['total_income'].describe().astype('int64')

count      21331
mean      165342
std        98313
min        20667
25%       107507
50%       142594
75%       195842
max      2265604
Name: total_income, dtype: int64

- низкий - доход не превышающий 107,507

- средний - в диапазоне от 107,507 до 142,594

- высокий - в диапазоне от 142,594 до 195,842

- очень высокий - превышающий 195,842

In [28]:
categorize_income_4= ['низкий', 'средний', 'высокий', 'очень высокий']

total_income['categorize'] = pd.qcut(total_income['total_income'],q=4, labels=categorize_income_4)
total_income.head()

,total_income,debt,categorize
0,253875,0,очень высокий
1,112080,0,средний
2,145885,0,высокий
3,267628,0,очень высокий
4,158616,0,высокий


Сгруппируем данные  по уровню дохода, посчитаем общее число заемщиков по каждому уровню и количество заемщиков, кто не вернул в срок кредит.

In [29]:
total_income=total_income.groupby('categorize')['debt'].agg(['count', 'sum']).rename(columns= {'count':'count_category','sum':'not_returned'})

Добавим в сводную таблицу столбец где буден виден процент таких заемщиков от общего числа

In [30]:
total_income['percent'] = round((total_income['not_returned']/total_income['count_category']*100).astype('float'),2)
total_income.sort_values(by='percent')

,count_category,not_returned,percent
categorize,,,
очень высокий,5332,381,7.15
низкий,5333,427,8.01
высокий,5216,444,8.51
средний,5450,480,8.81


**Вывод:** 

Данные показывают что уровень дохода не оказывает сильного влияния на возврат кредита в срок:

- Клиенты с очень высоким доходом имеют самый низкий процент просрочек.

- Клиенты с низким доходом возвращают кредит на 0,5%-0,8% чаще в срок, чем клиенты с высоким и средним доходом

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

Сгруппируем данные  по категориям целей на кредит, посчитаем общее число заемщиков по категориям и количество заемщиков, кто не вернул в срок кредит.

In [31]:
purpose_category=data.groupby('purpose_category')['debt'].agg(['count', 'sum']).rename(columns= {'count':'count_category','sum':'not_returned'})

Добавим в сводную таблицу столбик где буден виден процент таких заемщиков от общего числа

In [32]:
purpose_category['percent'] = round((purpose_category['not_returned']/purpose_category['count_category']*100).astype('float'),2)
purpose_category.sort_values(by='percent')

,count_category,not_returned,percent
purpose_category,,,
операции с недвижимостью,10751,780,7.26
проведение свадьбы,2313,183,7.91
получение образования,3988,369,9.25
операции с автомобилем,4279,400,9.35


**Вывод:** 

Данные показывают, что клиенты, выбиравшие разные цели для кредита по-разному погашают кредит в срок:

Клиенты, оформившие кредит для операций с недвижимостью на 2% лучше отдают кредиты по сравнению с клиентами, взявшими кредит на получение образования или на операции с автомобилем. Клиенты, оформившие кредит для проведения свадьбы отдают кредит во время примерно на том же уровне, что и клиенты взявшие кредит для операций с недвижимостью.

#### 3.5 Возможные причины появления пропусков в исходных данных.

Количество значений в столбцах days_employed и total_income различается. Значит, в данных есть пропущенные значения. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет.

Но можно предположить, что эти графы остались пустыми т.к. клиент никогда не работал (days_employed — общий трудовой стаж в днях) и не имеет ежемесячного дохода (total_income — ежемесячный доход) т.к. не работает в настоящий момент. Это можно проверить, выбрав строки с пустыми значениями total_income — ежемесячный доход и посмотреть, что стоит в income_type — тип занятости.

Просмотрев строки. Я пришла к выводу что, при выгрузке данных произошёл сбой, и пропуски появились по не известному нам принципу в одних и тех же строках в двух столбцах. 


#### 3.6 Почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

Так как пропуски в таблице присутствуют в столбцах с количественных переменными, их стоит заполнить медианным значением по столбцу так как на ее величину не влияют случайные выбросы.

### Шаг 4: общий вывод.


Перед проверкой гипотез был проведен обзор и предобработка данных:
- Проверка данных на пропущенные значения выявила пропуски в двух столбцах. Они были заполнены медианным значением.
- Затем, были удалены артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке и строки-дубликаты.
- Для проведения анализа о влиянии размера дохода на срок возвращения кредита, все клиенты были разделены на категории в зависимости от  размера их дохода.
- Так же для ответа на вопрос  как разные цели кредита влияют на его возврат в срок, необходимо было произвести категоризацию  целей кредита 


Мы проверили четыре гипотезы и установили:

1. Бездетные клиенты отдают кредиты на примерно 2 % лучше по сравнению с клиентами у которых есть дети. При этом само количество детей не сильно влияет на  процент возврата кредита вовремя.

2. Клиенты, которые официально оформляют брак более ответственно подходят и к срокам возврата кредита. Даже если их брак расторгнут или они овдовели.

3. Клиенты, имеющие очень высокий доход, отдают кредит в срок на 1-1,5% выше, чем клиенты со средним доходом, что возможно ожидаемо. Но как оказалось даже клиенты, с низким доходом возвращают кредит на 0,5%-0,8% чаще в срок, чем клиенты с высоким и средним доходом.
Этот результат мог оказаться иным, если бы не пропуски в данных.

4. Клиенты планирующие потрать заемные средства на недвижимость, более ответственно подходят к возврату кредита в срок чем клиенты желающие приобрести автомобиль или оплатить образование. Так же клиенты, вступающие в брак показали себя лучше чем автолюбители.

Из проведённого анализа можно сделать вывод, что клиент, состоящий в браке в прошлом или на сегодняшний день, не имеющий детей, является предпочтительным клиентом для получения кредита  с целью приобрести недвижимость.
